In [ ]:
# default_exp core

# Simple Neo4j HTTP API Client

> Simple http client to connect neo4j server

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import os,json,base64
import requests
class SimpleNeo4jHTTPAPIClient:
    @staticmethod
    def get_authtoken(userid, passwd):
        authstr='{}:{}'.format(userid, passwd)
        b64token=base64.b64encode(authstr.encode())
        strtoken=b64token.decode()
        return strtoken
    
    def __init__(self, url, db='neo4j', userid=None, passwd=None):
        self.url=url
        self.set_serverinfo(url)
        self.setdb(db)
        self.authtoken=None
        if userid is not None:
            self.authtoken=self.get_authtoken(userid, passwd)
        
    def set_serverinfo(self,url):
        resp=requests.get(url)
        obj=resp.json()
        self.bolt_routing=obj['bolt_routing']
        self.transaction=obj['transaction']
        self.bolt_direct=obj['bolt_direct']
        self.neo4j_version=obj['neo4j_version']
        self.neo4j_edition=obj['neo4j_edition']

    def setdb(self, db):
        self.db=db
        
    def execute_read_query(self, query, output_format=["row", "graph"]):
        url=self.transaction.format(databaseName=self.db) + '/commit'
        headers={
            "content-type": "application/json"
        }
        if self.authtoken is not None:
            headers['authorization']="Basic {}".format(self.authtoken)

        statement={
          "statements": [
            {
              "statement": query,
              "resultDataContents": output_format
            }
          ]
        }
        resp=requests.post(url, 
                           data=json.dumps(statement), 
                           headers=headers)

        return resp.json()

    
    
    def __repr__(self):
        return json.dumps({
            'classname':self.__class__.__name__,
            'url':self.url,
            'neo4j_version':self.neo4j_version,
            'neo4j_edition':self.neo4j_edition,
            'db':self.db,
            'auth': self.authtoken is not None
        })

In [ ]:
sample_output=json.loads("""
{
  "row": [
    [
      {
        "identifier": "10013",
        "license": "CCO 1.0",
        "chromosome": "X",
        "name": "HDAC6",
        "description": "histone deacetylase 6",
        "source": "Entrez Gene:210321",
        "url": "http://identifiers.org/ncbigene/10013"
      },
      {
        "action_score": 943,
        "homology": 0,
        "experiments": 483,
        "textmining_transferred": 642,
        "combined_score": 923,
        "source": "String",
        "version": "v11.5",
        "license": "CC BY 4.0",
        "experiments_transferred": 86,
        "database": 600,
        "unbiased": "False",
        "action": "catalysis;reaction",
        "database_transferred": 0,
        "textmining": 0
      },
      {
        "identifier": "3326",
        "license": "CCO 1.0",
        "chromosome": "6",
        "name": "HSP90AB1",
        "description": "heat shock protein 90 alpha family class B member 1",
        "source": "Entrez Gene:210321",
        "url": "http://identifiers.org/ncbigene/3326"
      }
    ]
  ],
  "meta": [
    [
      {
        "id": 6967,
        "type": "node",
        "deleted": false
      },
      {
        "id": 7460908,
        "type": "relationship",
        "deleted": false
      },
      {
        "id": 73199,
        "type": "node",
        "deleted": false
      }
    ]
  ],
  "graph": {
    "nodes": [
      {
        "id": "6967",
        "labels": [
          "Gene"
        ],
        "properties": {
          "identifier": "10013",
          "license": "CCO 1.0",
          "chromosome": "X",
          "name": "HDAC6",
          "description": "histone deacetylase 6",
          "source": "Entrez Gene:210321",
          "url": "http://identifiers.org/ncbigene/10013"
        }
      },
      {
        "id": "73199",
        "labels": [
          "Gene"
        ],
        "properties": {
          "identifier": "3326",
          "license": "CCO 1.0",
          "chromosome": "6",
          "name": "HSP90AB1",
          "description": "heat shock protein 90 alpha family class B member 1",
          "source": "Entrez Gene:210321",
          "url": "http://identifiers.org/ncbigene/3326"
        }
      }
    ],
    "relationships": [
      {
        "id": "7460908",
        "type": "INTERACTS_GiG2",
        "startNode": "73199",
        "endNode": "6967",
        "properties": {
          "action_score": 943,
          "homology": 0,
          "experiments": 483,
          "textmining_transferred": 642,
          "combined_score": 923,
          "source": "String",
          "version": "v11.5",
          "license": "CC BY 4.0",
          "experiments_transferred": 86,
          "database": 600,
          "unbiased": "False",
          "action": "catalysis;reaction",
          "database_transferred": 0,
          "textmining": 0
        }
      }
    ]
  }
}""")

## Tests

In [ ]:
# Connect to a neo4j server without auth info
client=SimpleNeo4jHTTPAPIClient(url='http://localhost:7474')
client

{"classname": "SimpleNeo4jHTTPAPIClient", "url": "http://localhost:7474", "neo4j_version": "4.4.3", "neo4j_edition": "community", "db": "neo4j", "auth": false}

In [ ]:
result=client.execute_read_query('match (n) return count(n);')
result

{'errors': [{'code': 'Neo.ClientError.Security.Unauthorized',
   'message': 'No authentication header supplied.'}]}

In [ ]:
assert len(result['errors']) > 0

In [ ]:
# Connect to a neo4j server with auth info
client=SimpleNeo4jHTTPAPIClient(url='http://localhost:7474', userid='neo4j',passwd='test')
client

{"classname": "SimpleNeo4jHTTPAPIClient", "url": "http://localhost:7474", "neo4j_version": "4.4.3", "neo4j_edition": "community", "db": "neo4j", "auth": true}

In [ ]:
result=client.execute_read_query('match (n) return count(n);')
result

{'results': [{'columns': ['count(n)'],
   'data': [{'row': [85041],
     'meta': [None],
     'graph': {'nodes': [], 'relationships': []}}]}],
 'errors': []}

In [ ]:
assert len(result['errors']) == 0